# Actividad 2 - Clasificación
# Universidad Europea de Madrid
Inteligencia Artificial<br>
Isabel Sutil<br>
M31<br>
Jorge Javier Castilla Coello<br>
Geronimo Basso Sosa<br>
Luis Cabello Casquete<br>
Paula Sáenz de Santa María Diez<br>
Ana Esteban González<br>




# Apartado 0
Modificar el dataset para que cada vez que en "prec" se encuente el valor 0,0 se considere que no llueve, para los demas valores se va a considerar que si ha llovido

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.base import clone
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import numpy as np
from tabulate import tabulate
from scipy import stats

In [ ]:
# Cargamos el fichero necesario para analizar los modelos.
ruta_archivo = 'valores-climatologicos.csv'
fdata = pd.read_table(ruta_archivo, sep=',')

In [ ]:
# Utilizando una función lambda, para cada valor de ´prec´cambiamos su valor a No si la precipitación es 0
# cambiamos el valor a Yes si la precipitación es disitnto de 0.
fdata['prec'] = fdata['prec'].apply(lambda x: '0' if x == 0.0 else '1') # 0 si no llovio, 1 si llovio.
states = ['1387E', '1387', '1393', '1351', '1400', '1437O', '1473A', '1428', '1475X', '1505', '1484', '1484C', '1495']
# Creating a map where each unique state is assigned a unique number
state_map = {state: idx for idx, state in enumerate(sorted(set(states)))}
fdata['indicativo'] = fdata['indicativo'].apply(lambda x: state_map[x])

In [ ]:
# Visualización de las primeras filas del dataset.
print(fdata.head())

# Apartado 1
Análisis exploratorio

In [ ]:
# Verificamos si hay hay valores nulos que faltan en nuestro dataset.
print(fdata.isna().any())

In [ ]:
# Convertir la columna 'prec' de 'object' a 'categorical'
fdata['prec'] = pd.Categorical(fdata['prec'])
print(fdata.dtypes)

# Function to train models.

In [134]:
from sklearn.model_selection import train_test_split

X = fdata.drop('prec', axis=1)  # Features
y = fdata['prec']  # Variable de salida

# División en 80/20 para entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Imprimir tamaños de los conjuntos
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de validación:", len(X_val))

Tamaño del conjunto de entrenamiento: 9228
Tamaño del conjunto de validación: 2308


In [135]:
# Crear conjuntos de entrenamiento y prueba
conjunto_train = pd.concat([X_train, y_train], axis=1)
conjunto_val = pd.concat([X_val, y_val], axis=1)


# Creamos dos conjuntos para poner las predicciones del train y test
conjunto_train_eval = conjunto_train.copy()
conjunto_validation_eval = conjunto_val.copy()


In [136]:
# Método de control (k-fold cross-validation)
metodo_control_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Apartado 2
Análisis predictivo con Regresión Lineal

In [137]:
# Definir el modelo de regresión logística
logreg_model = LogisticRegression()

# Crear un pipeline con escalado de características
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Escalar las características
    ('model', logreg_model)        # Modelo de regresión logística
])


# Realizar validación cruzada y obtener predicciones
y_pred_cv = cross_val_predict(pipeline, X_train, y_train, cv=metodo_control_cv, method='predict_proba')
y_val_pred_raw = cross_val_predict(pipeline, X_train, y_train, cv=metodo_control_cv)

In [138]:
# Obtener todos los modelos de regresión logística entrenados durante la validación cruzada
modelo_regresionLogistica_1 = [clone(pipeline).fit(X_train.iloc[train_idx], y_train.iloc[train_idx]) for train_idx, _ in metodo_control_cv.split(X_train, y_train)]

# Obtener los coeficientes de cada modelo entrenado
coeficientes_cv = [modelo.named_steps['model'].coef_ for modelo in modelo_regresionLogistica_1]

# Imprimir información sobre el modelo
for i, (modelo, coeficientes) in enumerate(zip(modelo_regresionLogistica_1, coeficientes_cv)):
    print(f"\nModelo de regresión logística entrenado en el pliegue {i + 1}:")
    print(f"Peso de X1 y X2:", coeficientes)
    precision = modelo.score(X_train, y_train)
    print("Precisión en el step:", precision)


Modelo de regresión logística entrenado en el pliegue 1:
Peso de X1 y X2: [[ 0.33349958 -0.01081902 -0.17954089  1.26919946 -1.65962565  0.25091472
  -1.00652378  1.25473612  1.36654868 -1.46180238]]
Precisión en el step: 0.7483745123537061

Modelo de regresión logística entrenado en el pliegue 2:
Peso de X1 y X2: [[ 0.33205953  0.00519328 -0.2208929   1.27317739 -1.60844928  0.24451122
  -1.00455596  1.29385836  1.33012063 -1.42615725]]
Precisión en el step: 0.7488079757260512

Modelo de regresión logística entrenado en el pliegue 3:
Peso de X1 y X2: [[ 0.33742835  0.00983295 -0.23606851  1.28722886 -1.62983584  0.24427861
  -1.00377923  1.27455301  1.35808086 -1.46770763]]
Precisión en el step: 0.7496749024707412

Modelo de regresión logística entrenado en el pliegue 4:
Peso de X1 y X2: [[ 0.31971982  0.0355363  -0.17692087  1.2620021  -1.66003405  0.23672905
  -0.98912166  1.24567973  1.33647663 -1.44913777]]
Precisión en el step: 0.7493498049414824

Modelo de regresión logística e

In [ ]:
train_model(LogisticRegression())

# Apartado 3
Análisis predictivo con KNN

In [ ]:
train_model(KNeighborsClassifier(n_neighbors=5))

# Apartado 4
Análisis predictivo con un árbol de decisión

# Apartado 5
Análisis predictivo con SVM

# Apartado 6
Análisis predictivo con Naive Bayes.

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Apartado 7
Análisis comparativo de todos los modelos en train y en validation